# Human Population Size on an Earth-like Planet--a Computer Experiment

In [33]:
from pathlib import Path
import sys

import networkx as nx
from pyvis.network import Network
import numpy as np
from scipy import stats
from pert import PERT
import plotly.express as px


In [2]:
rng = np.random.default_rng()

The response variable is total population size after SIMULATION_YEARS.

In [ ]:
NUM_WORLD_LOCATIONS = 100
SIMULATION_YEARS = 10000
INITIAL_POPULATION_PROPORTION = 0.0001
# square miles 
TOTAL_LAND_AREA = 57268900 
# The average maximum sustainable population density 
# (per square mile) on the planet for a hunter-gatherer society.
# https://en.wikipedia.org/wiki/Hunter-gatherer#:~:text=One%20group%2C%20the%20Chumash%2C%20had,21.6%20persons%20per%20square%20mile.
INITIAL_MAX_POP_DENSITY = 20
INITIAL_AVG_CARRYING_CAPACITY_PER_LOCATION = (INITIAL_MAX_POP_DENSITY * TOTAL_LAND_AREA) / NUM_WORLD_LOCATIONS

print(
    f"INITIAL_AVG_CARRYING_CAPACITY_PER_LOCATION: {INITIAL_AVG_CARRYING_CAPACITY_PER_LOCATION}"
)

BIOMES = [
    "tropical and subtropical moist broadleaf forests",
    "tropical and subtropical dry broadleaf forests",
    "tropical and subtropical coniferous forests",
    "temperate broadleaf and mixed forests",
    "temperate coniferous forests",
    "boreal forests/taiga",
    "tropical and subtropical grasslands, savannas, and shrublands",
    "temperate grasslands, savannas, and shrublands",
    "flooded grasslands and savannas",
    "montane grasslands and shrublands",
    "tundra",
    "Mediterranean forests, woodlands, and scrub or sclerophyll forests",
    "deserts and xeric shrublands",
    "mangrove"
]

In [ ]:
biomes = Network(
    directed=False,
    neighborhood_highlight=False, 
    select_menu=True, 
    filter_menu=True,
    cdn_resources="in_line"
)

pre_biomes = nx.Graph()

# https://stackoverflow.com/a/47555011/8423001
nodes_and_biomes_dict = {node_id: BIOMES[node_id] for node_id in range(len(BIOMES))}

pre_biomes.add_nodes_from(
    [(node, {"biome": attribute}) 
        for (node, attribute) 
        in nodes_and_biomes_dict.items()
    ] 
)

pre_biomes.add_edges_from(
    [
        (0, 1),
        (0, 2),
        (0, 3),
        (0, 4),
        (0, 6),
        (0, 9),
        (0, 12),
        (0, 13),
        (1, 2),
        (1, 6),
        (1, 8),
        (1, 9),
        (1, 12),
        (1, 13),
        (2, 3),
        (2, 4),
        (2, 6),
        (2, 8),
        (2, 9),
        (2, 12),
        (2, 13),
        (3, 4),
        (3, 5),
        (3, 6),
        (3, 7),
        (3, 8),
        (3, 9),
        (3, 11),
        (3, 12),
        (3, 13),
        (4, 5),
        (4, 6),
        (4, 7),
        (4, 8),
        (4, 9),
        (4, 10),
        (4, 11),
        (4, 12),
        (4, 13),
        (5, 7),
        (5, 8),
        (5, 9),
        (5, 10),
        (6, 7),
        (6, 8),
        (6, 9),
        (6, 12),
        (6, 13),
        (7, 8),
        (7, 9),
        (7, 11),
        (7, 12),
        (8, 9),
        (8, 11),
        (8, 12),
        (8, 13),
        (9, 11),
        (9, 12),
        (9, 13),
        (11, 12),
        (11, 13),
        (12, 13)
    ]
)

In [ ]:
# biomes.from_nx(nx_graph=pre_biomes)
# biomes.show("biomes.html")

In [ ]:
world = Network(
    directed=True,
    neighborhood_highlight=True, 
    select_menu=True, 
    filter_menu=True,
    cdn_resources="in_line"
)

pre_world = nx.connected_watts_strogatz_graph(
    n=NUM_WORLD_LOCATIONS,
    k=5,
    p=0.5
)

In [ ]:
for (v1, v2, weight) in pre_world.edges.data('weight'):
    # https://trenton3983.github.io/files/projects/2020-05-21_intro_to_network_analysis_in_python/2020-05-21_intro_to_network_analysis_in_python.html
    # https://stackoverflow.com/questions/40128692/networkx-how-to-add-weights-to-an-existing-g-edges

    # Here, the weights represent the ease of travelling between nodes.
    # A high weight indicates that travel is easy.
    pre_world[v1][v2]["weight"] = stats.expon.rvs(scale=1)

# Make the graph directed to indicate
# allowable population movements.
pre_world = pre_world.to_directed()

In [ ]:
# Skip the last row because we only care about
# the upper triangle exclusive of th main diagonal
# of the adjacency matrix.
for v1 in range(NUM_WORLD_LOCATIONS - 1):
    for v2 in range(v1 + 1, NUM_WORLD_LOCATIONS):
        current_edge_data = pre_world.get_edge_data(v1, v2)
        if current_edge_data is None:
            # There is no need to update current_weight.
            continue

        # Extract weight attribute
        current_weight = current_edge_data["weight"]
        if current_weight < 1:
            # Make the ease of travel different
            # for one of the edges connecting the same
            # pair of nodes to simulate ocean currents.
            pre_world[v1][v2]["weight"] = stats.expon.rvs(scale=0.2)


In [ ]:
pre_world_betweenness_centralities = nx.betweenness_centrality(
    G=pre_world,
    weight="weight"
)

pre_biomes_betweenness_centralities = nx.betweenness_centrality(
    G=pre_biomes
)

# Get a node with a maximal betweenness centrality.
# This node will hold our starting population.
# https://stackoverflow.com/a/280156/8423001
starting_node = max(
    pre_world_betweenness_centralities, 
    key=pre_world_betweenness_centralities.get
)

starting_node_biome_id = max(
    pre_biomes_betweenness_centralities, 
    key=pre_biomes_betweenness_centralities.get
)

starting_node_biome = BIOMES[starting_node_biome_id]

In [ ]:
sorted(list(pre_biomes_betweenness_centralities.values()))

In [ ]:
# https://stackoverflow.com/a/3071441/8423001
(
    stats.rankdata(
        a=list(pre_biomes_betweenness_centralities.values()),
        method="max"
    )
    # Because the ranks start at 1 but Python is 0-indexed,
    # subtract 1.
    - 1
)

In [ ]:
sum(np.array(list(pre_biomes_betweenness_centralities.values())) <= 0.01)

In [ ]:
stats.rankdata(
        a=[-2, 0, 3, 3, 3],
        method="max"
    )

In [ ]:
np.arange(5) * 10


In [ ]:
def stochastic_func(
    x,
    b,
    corr
):
    rng = np.random.default_rng()
    std_x = np.std(x)
    if std_x == 0:
        y = rng.choice(np.arange(b + 1))
    else:
        x_normalized = (x - np.mean(x))/np.std(x)
 
        y_normalized = corr * x_normalized
        std_ints = np.std(np.arange(b + 1))
        mean_ints = (1 + b)/2
        y = y_normalized * std_ints + mean_ints
    return y

Copula Stuff

In [230]:
def gaussian_copula(*args, **kwargs):
    # https://en.wikipedia.org/wiki/Copula_(probability_theory)#Gaussian_copula
    # Arguments provided via position should be 
    # real numbers in [0, 1].  
    # kwargs should contain a key=value combination
    # where the key is cov.
    #
    # The multivariate_normal.cdf returns nan when the corresponding
    # probability law is at least two dimensional and at least one of 
    # the values supplied to x is -inf.  However, we think that it is
    # reasonable for it just to return 0 instead of nan.
    x = stats.norm.ppf(q=args)
    if (x == float("-inf")).any():
        cdf = 0
    else:
        cdf = stats.multivariate_normal.cdf(
            x=stats.norm.ppf(q=args),
            mean=np.zeros(shape=len(args)),
            allow_singular=True,
            **kwargs      
        )

    return cdf

In [232]:
def bivariate_discrete_copula_pmf(C, u:int, v:int, R:int, S:int, **kwargs) -> float:
    """Get the value of the copula probability mass function
    at (u, v) for the copula function C.

    source: https://doi.org/10.1515/demo-2020-0022
    see: equation 7.1
    """
    if (u < 0) or (u > (R - 1)):
        raise ValueError("u must be in {0, 1, ..., R - 1}")
    if (v < 0) or (v > (S - 1)):
        raise ValueError("v must be in {0, 1, ..., S - 1}")

    pmf = C((u + 1)/R, (v + 1)/S, **kwargs) \
        - C(u/R, (v + 1)/S, **kwargs) \
        - C((u + 1)/R, v/S, **kwargs) \
        + C(u/R, v/S, **kwargs)
    
    if (pmf < (0 - sys.float_info.epsilon)) or (pmf > (1 + sys.float_info.epsilon)):
        raise RuntimeError("C appears to be an invalid copula.")
    
    return pmf

In [250]:
# Draw u from a discrete uniform distribution
# with parameters 0 and R - 1.
# Choose v based on the conditional PMF for 
# the value of u that was picked.
R = 20
S = 12
corr_mat = np.array([
    [1, -0.75],
    [-0.75, 1]
])
times = 100
us_and_vs = np.empty(shape=(times, 2))
for j in range(times):
    u = rng.integers(low=0, high=R, endpoint=False)
    probs = np.empty(shape=S)
    for possible_v in range(S):
        probs[possible_v] = bivariate_discrete_copula_pmf(C=gaussian_copula, u=u, v=possible_v, R=R, S=S, cov=corr_mat)
        # print(f"probs[possible_v]: {probs[possible_v]},   u: {u}   v: {possible_v}")
    probs_as_ints = (probs * (2 ** (32 - 1))).astype(np.int32)
    probs_as_probs = (probs_as_ints / probs_as_ints.sum())
    v = rng.choice(a=S, p=probs_as_probs)
    us_and_vs[j, 0] = u
    us_and_vs[j, 1] = v

In [251]:
us_and_vs[1:30, :]

array([[ 0., 11.],
       [ 6.,  6.],
       [19.,  0.],
       [13.,  6.],
       [ 9.,  9.],
       [10., 11.],
       [ 3., 11.],
       [16., 10.],
       [ 6.,  5.],
       [16.,  4.],
       [ 2., 11.],
       [ 7.,  9.],
       [12.,  1.],
       [19.,  0.],
       [15.,  1.],
       [11.,  2.],
       [ 3., 10.],
       [13.,  9.],
       [19.,  0.],
       [12., 10.],
       [12.,  8.],
       [13.,  7.],
       [ 3.,  9.],
       [ 5.,  9.],
       [ 2.,  7.],
       [ 0., 10.],
       [ 5., 10.],
       [13.,  5.],
       [16.,  5.]])

In [252]:
px.density_heatmap(x=us_and_vs[:, 0], y=us_and_vs[:, 1], marginal_x="histogram", marginal_y="histogram")

In [ ]:
(probs * (2 ** (32 - 1))).astype(np.int32) / 

In [ ]:
(probs / probs.sum())

In [ ]:
# The copula itself can be found by applying Sklar's Theorem.
# That is,  

In [ ]:
0 - sys.float_info.epsilon

In [ ]:
stochastic_func(
    x=np.array([7, 8, 1, 3, 2, 5, 4, 6]),
    b=4,
    corr=0.5
)

In [ ]:
def surjective_stochastic_func(
    x,
    b,
    _lambda
):
    """
    Args:
        x: numpy.ndarray. A one-dimensional numpy.ndarray
            whose elements are elements in the domain of 
            the form [x0, x1, x2, . . .]
        b: int. The least upper bound of the range.
        _lambda: float. Should be greater than 0.

    Returns:
        A numpy.ndarray whose elements are integers of the form
            [f(x0), f(x1), f(x2), . . .]
    """
    
    num_x_obs = len(x)
    # y will hold the outputs of the function
    y = np.empty(shape=num_x_obs, dtype=np.int32)
    is_surjective = False
    # How high x is should influence how high
    # p is for drawing from a binomial distribution
    # to finally get y = f(x).

    # Repeatedly generate possible realizations for y
    # until surjectivity is achieved.
    while is_surjective is False:
        # Initialize variable for indexing into y.
        j = 0
        for i, x_i in enumerate(x):
            # Sample from a PERT distribution
            # with most likely value p_most_likely
            p_most_likely = (x_i + 1) / (num_x_obs + 2)

            p = PERT(
                min_val=0,
                ml_val=p_most_likely,
                max_val=1,
                lamb=_lambda
            ).rvs(size=1)
            # This binomial random variable is supported from 0
            # to b - 1.  However, for all i, f(x_i) should be anything
            # between 1 and b.  Thus, we add 1 at the end.
            y[i] = stats.binom.rvs(n=b - 1, p=p, size=1).item() + 1
        
        # Test for surjectivity after building out y
        is_surjective = bool(np.isin(element=np.arange(1, b + 1, 1), test_elements=y).all())

    return y

In [ ]:
np.isin(element=np.arange(1, 9 + 1, 1), test_elements=np.array([3, 7, 4, 7, 2, 6, 4, 4])).all()

In [ ]:
surjective_stochastic_func(
    x = np.array([7, 8, 1, 3, 2, 5, 4, 6]), 
    b = 4,
    _lambda = 10
)

In [ ]:
def get_correlated_ranks(
    x_ranks,
    num_y_obs
):
    """Suppose that the ranks of observations for some variable X (x_ranks)
    are available.  We assume that X has a high Spearman's correlation 
    coefficient with an unobserved variable Y.  We would like to sample
    from the conditional distribution of Rank(Y)|Rank(X).  
    
    
    samples have been taken from the joint distribution
    of ordinal or quantitative factors X and Y.  
    
    For each rank in x_ranks, return 
    
    Given the ranks of observations for some variable X (x_ranks),
    return 
    randomly generated ranks for observations of some other Y.

    The ranks for observations of Y are generated such that
    the Spearman rank correlation
    coefficient between X and Y is high.

    Args:
        x_ranks: list. This should be non-empty.
            The maximum element should be no larger than
            the length of the list.

    Returns:
        list. The elements of the returned list correspond to 
            ranks for observations of Y.  The returned list
            should have a length of len(x_ranks) because we
            assume that for every observation of X, there is
            exactly one observation of Y.
    """

    num_x_obs = len(x_ranks)
    num_y_obs = num_x_obs

    y_ranks = []
    for x_rank in x_ranks:
        # Sample from a PERT distribution
        # with most likely value p_most_likely
        p_most_likely = (x_rank + 1) / (num_x_obs + 2)

        p = PERT(
            min_val=0,
            ml_val=p_most_likely,
            max_val=1
        ).rvs(size=1)
        # This binomial random variable can be anything from 0
        # to num_y_obs.  However, y_ranks should be anything
        # between 1 and num_y_obs.
        # The minus 1 ensures us that 
        y_ranks.append(stats.binom.rvs(n=num_y_obs - 1, p=p, size=1).item() + 1)
    
    return y_ranks

In [ ]:
thingys = get_correlated_ranks(x_ranks=[3, 1.5, 1.5, 4.5, 4.5, 6], num_y_obs=3)
thingys

In [ ]:
# We plan on assigning biomes to the nodes in our world.
# But, we must consider that some biomes are more likely
# to be connected.  Thus, we assign the biomes randomly
# while taking account of the betweenness centralities.
# With probability 0.5, we assign neighbors the same
# biome, while with probability 0.5, we assign neighbors
# a new biome of similar betweenness centrality.
1.0 / NUM_WORLD_LOCATIONS
sorted(pre_biomes_betweenness_centralities.values())
# Given a value of the ECDF of pre_world_betweenness_centralities
# generate an appropriately positioned random rank
# within pre_biomes_betweenness_centralities.
# First, rank the pre_world_betweenness_centralities.
sorted(pre_world_betweenness_centralities.values())
# Second, find the find the value of the ECDF for each rank.

In [ ]:
np.quantile(
    a=list(pre_world_betweenness_centralities.values()),
    q=0.5
)

In [ ]:
for id in pre_biomes.neighbors(starting_node_biome_id):
    print(id)

In [ ]:
# Loop through nodes and set initial parameters.
for node in nx.nodes(G=pre_world):
    nx.set_node_attributes(
        G=pre_world, 
        values={node: {"carrying_capacity": 1000000}}
    )

In [ ]:
world.from_nx(nx_graph=pre_world, show_edge_weights=True)
world.show("world.html")